In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm.notebook import tqdm

# Base URL of the form
url = "https://www.dmr.nd.gov/oilgas/findwellsvw.asp"

# Township values
township_values = [str(i) for i in range(129, 165)]
all_wells = []

# Loop through each Township with progress bar
for t_value in tqdm(township_values, desc="Scraping Townships"):
    # POST data
    data = {
        "ddmTownship": t_value,
        "B1": "Submit"  # the submit button name/value
    }
    
    # Send POST request
    response = requests.post(url, data=data)
    
    if response.status_code != 200:
        print(f"Failed for Township {t_value}")
        continue
    
    soup = BeautifulSoup(response.text, "html.parser")
    
    # Find the well table
    well_table = None
    for tbl in soup.find_all("table"):
        headers = [th.get_text(strip=True) for th in tbl.find_all("th")]
        if "API No" in headers:
            well_table = tbl
            break
    
    if not well_table:
        print(f"No table found for Township {t_value}")
        continue
    
    # Extract rows
    rows = well_table.find_all("tr")[1:]  # skip header
    for row in rows:
        cells = row.find_all("td")
        if len(cells) != 11:
            continue
        well = {
            "File No": cells[0].get_text(strip=True),
            "CTB No": cells[1].get_text(strip=True),
            "API No": cells[2].get_text(strip=True),
            "Well Type": cells[3].get_text(strip=True),
            "Well Status": cells[4].get_text(strip=True),
            "Status Date": cells[5].get_text(strip=True),
            "DTD": cells[6].get_text(strip=True),
            "Location": cells[7].get_text(strip=True),
            "Operator": cells[8].get_text(strip=True),
            "Well Name": cells[9].get_text(strip=True),
            "Field": cells[10].get_text(strip=True)
        }
        all_wells.append(well)

# Convert to DataFrame
df = pd.DataFrame(all_wells)
print(f"Scraped {len(df)} wells in total.")

# Save to CSV
output_file = "ND_wells.csv"
df.to_csv(output_file, index=False)
print(f"Saved to {output_file}")




Scraping Townships:   0%|          | 0/36 [00:00<?, ?it/s]

Scraped 43125 wells in total.
Saved to ND_wells.csv
